In [1]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlrd
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from nilearn.connectome import ConnectivityMeasure
from pandas import DataFrame
import scipy as sc
from scipy import io
from scipy.stats import pearsonr
from os.path import join, exists, dirname
from glob import glob
from brainspace import gradient
import nibabel as nib

In [ ]:
# Start characterize subytpe using Funtaional Random Forest result

In [ ]:
demo = pd.read_excel(join(path_data, 'abide_func_fn_abideII_sorted.xlsx'), sheet_name='Sheet1', skiprows=0)

sub_list = demo['ID']
label2 = demo['Group']
label = demo['DX_GROUP']
site_id = demo['Site']
site_label = demo['SITE_Label']
Age = demo['Age']
FD = demo['MeanFD_Jenkinson'] # = func_mean_fd
IQ = demo['FIQ']
sex = demo['Sex']

ASD_index = np.where(label == 1)[0]                
TD_index = np.where(label == 2)[0]
Total_index = np.concatenate((ASD_index,TD_index)) 
sorted_idx = np.concatenate((ASD_index,TD_index), axis = 0)

site_label_sorted = np.array(site_label)[sorted_idx]

male_idx = np.where(sex=='M')[0]
label_male = np.array(label[male_idx])
ASD_male_idx = np.array(list(set(np.where(label == 1)[0]) & set(male_index)))
TD_male_idx = np.array(list(set(np.where(label == 2)[0]) & set(male_index)))
sorted_male_idx = np.concatenate((ASD_male_idx, TD_male_idx), axis=0)

whole2male_idx_ABD2 = np.array([np.where(sorted_idx==sorted_male_idx[i])[0][0] for i in range(len(sorted_male_idx))])

print('Total : ', len(Total_index), len(sorted_male_idx))
print('ASD : ', len(ASD_index), len(ASD_male_idx))
print('TD : ', len(TD_index), len(TD_male_idx))

In [ ]:
# Load reg out mat

subj_num = 103
grad_num = 1
Thresh = 'top50' # top50
valid = '_Valid' # '_Valid' ''

main_result = False

if main_result:
    regout_mat = sc.io.loadmat(join(path_data,f'Gradient{grad_num}_Surf_pearson_sorted_regout_gender_Mean_har_n{subj_num}_{Thresh}_ABD2.mat'))

    PCs_sorted_reg_out = regout_mat['total_group_data'][:,:360]

    grp_ASD_PCs_reg_out = PCs_sorted_reg_out[:len(ASD_index),:]
    grp_TD_PCs_reg_out = PCs_sorted_reg_out[len(ASD_index):,:]

    # Load FRF subtype result

    folder = f'Gradient{grad_num}_pearson_MeanHar_n{subj_num}_{Thresh}_1000iter_1perm_rtoz_ABD_2_FeaSel{valid}_output'
    FRF_outbputs_DMN_pcs = sc.io.loadmat(join(path_mat,f'{folder}/subgroup_community_assignments.mat'))
else:
    Topk='Top50'
    regout_mat = sc.io.loadmat(join(path_data,f'Gradient{grad_num}_Surf_pearson_sorted_regout_gender_Mean_har_n{subj_num}_{Thresh}_ABD2.mat'))
    PCs_sorted_reg_out = regout_mat['total_group_data'][whole2male_idx_ABD2,:360] # whole2male_idx_ABD2
    label = regout_mat['total_group_data'][whole2male_idx_ABD2,360] # whole2male_idx_ABD2
    grp_ASD_PCs_reg_out = PCs_sorted_reg_out[np.where(label==1)[0], :] # np.where(label==1)[0] 
    grp_TD_PCs_reg_out = PCs_sorted_reg_out[np.where(label==2)[0], :] # np.where(label==2)[0] 
    
    ASD_label_idx = np.where(label==1)[0]
    TD_label_idx = np.where(label==2)[0]

    # Load FRF subtype result
    folder = f'Gradient{grad_num}_pearson_MeanHar_{Thresh}_100iter_1perm_rtoz_FeaSel_split06_valid_male_ABD2_output'
    FRF_outbputs_DMN_pcs = sc.io.loadmat(join(path_mat,'review','Top50_main',f'{folder}/subgroup_community_assignments.mat'))    

# subgroup_order
subgroup_order = []
for i in range(len(male_index)):
    subgroup_order.append(FRF_outbputs_DMN_pcs['subgroup_community_assignments'][i][0][0])
    
# subgroup_order
ASD1 = np.where(np.array(subgroup_order)=='G1_1')[0].shape[0]
ASD2 = np.where(np.array(subgroup_order)=='G1_2')[0].shape[0]
ASD3 = np.where(np.array(subgroup_order)=='G1_3')[0].shape[0]
ASD4 = np.where(np.array(subgroup_order)=='G1_4')[0].shape[0]

TD1 = np.where(np.array(subgroup_order)=='G2_1')[0].shape[0]
TD2 = np.where(np.array(subgroup_order)=='G2_2')[0].shape[0]
TD3 = np.where(np.array(subgroup_order)=='G2_3')[0].shape[0]
TD4 = np.where(np.array(subgroup_order)=='G2_4')[0].shape[0]
TD5 = np.where(np.array(subgroup_order)=='G2_5')[0].shape[0]

# Assign subtype idx
ASD_DMN_pc_sub1_idx = np.array(FRF_outbputs_DMN_pcs['subgroup_sorting_orders'][0][0].reshape(-1,),dtype='int64')[:ASD1]-1
ASD_DMN_pc_sub2_idx = np.array(FRF_outbputs_DMN_pcs['subgroup_sorting_orders'][0][0].reshape(-1,),dtype='int64')[ASD1 : ASD1+ASD2]-1
ASD_DMN_pc_sub3_idx = np.array(FRF_outbputs_DMN_pcs['subgroup_sorting_orders'][0][0].reshape(-1,),dtype='int64')[ASD1+ASD2:ASD1+ASD2+ASD3]-1
ASD_DMN_pc_sub4_idx = np.array(FRF_outbputs_DMN_pcs['subgroup_sorting_orders'][0][0].reshape(-1,),dtype='int64')[ASD1+ASD2+ASD3:]-1


TD_DMN_pc_sub1_idx = np.array(FRF_outbputs_DMN_pcs['subgroup_sorting_orders'][1][0].reshape(-1,),dtype='int64')[:TD1]-1
TD_DMN_pc_sub2_idx = np.array(FRF_outbputs_DMN_pcs['subgroup_sorting_orders'][1][0].reshape(-1,),dtype='int64')[TD1 : TD1+TD2]-1   
TD_DMN_pc_sub3_idx = np.array(FRF_outbputs_DMN_pcs['subgroup_sorting_orders'][1][0].reshape(-1,),dtype='int64')[TD1+TD2 :TD1+TD2+TD3]-1  
TD_DMN_pc_sub4_idx = np.array(FRF_outbputs_DMN_pcs['subgroup_sorting_orders'][1][0].reshape(-1,),dtype='int64')[TD1+TD2+TD3 :]-1  


ASD_DMN_pc_sub1 = grp_ASD_PCs_reg_out[ASD_DMN_pc_sub1_idx,:]
ASD_DMN_pc_sub2 = grp_ASD_PCs_reg_out[ASD_DMN_pc_sub2_idx,:]
ASD_DMN_pc_sub3 = grp_ASD_PCs_reg_out[ASD_DMN_pc_sub3_idx,:]
ASD_DMN_pc_sub4 = grp_ASD_PCs_reg_out[ASD_DMN_pc_sub4_idx,:]


TD_DMN_pc_sub1 = grp_TD_PCs_reg_out[TD_DMN_pc_sub1_idx,:]
TD_DMN_pc_sub2 = grp_TD_PCs_reg_out[TD_DMN_pc_sub2_idx,:]
TD_DMN_pc_sub3 = grp_TD_PCs_reg_out[TD_DMN_pc_sub3_idx,:]
TD_DMN_pc_sub4 = grp_TD_PCs_reg_out[TD_DMN_pc_sub4_idx,:]



print(ASD_DMN_pc_sub1.shape)
print(ASD_DMN_pc_sub2.shape)
print(ASD_DMN_pc_sub3.shape)
print(ASD_DMN_pc_sub4.shape)


print(' ')
print(TD_DMN_pc_sub1.shape)
print(TD_DMN_pc_sub2.shape)
print(TD_DMN_pc_sub3.shape)
print(TD_DMN_pc_sub4.shape)



In [549]:
# ADOS_Score assignment
label = np.array(demo['DX_GROUP'])

ADOS_Total = np.nan_to_num(np.array(demo['ADOS_G_TOTAL']), nan = -1e-16) # -9999 -> -1, nan -> -2
ADOS_Total = np.where(ADOS_Total == -999, -1e-16, ADOS_Total)
ADOS_Total
ADOS_Total_sorted = ADOS_Total[sorted_idx]

ADOS_comm = np.nan_to_num(np.array(demo['ADOS_G_COMM']), nan = -1e-16)
ADOS_comm = np.where(ADOS_comm == -999, -1e-16, ADOS_comm)
ADOS_comm_sorted = ADOS_comm[sorted_idx]

ADOS_social =  np.nan_to_num(np.array(demo['ADOS_G_SOCIAL']), nan = -1e-16)
ADOS_social = np.where(ADOS_social == -999, -1e-16, ADOS_social)
ADOS_social_sorted = ADOS_social[sorted_idx]

ADOS_behav =  np.nan_to_num(np.array(demo['ADOS_G_STEREO_BEHAV']), nan = -1e-16)
ADOS_behav = np.where(ADOS_behav == -999, -1e-16, ADOS_behav)
ADOS_behav_sorted = ADOS_behav[sorted_idx]

age_sorted = np.array(Age)[sorted_idx]
iq_sorted = np.array(IQ)[sorted_idx]
sex_sorted = np.array(sex)[sorted_idx]

x = ADOS_Total_sorted[whole2male_idx_ABD2]
y = ADOS_comm_sorted[whole2male_idx_ABD2]
z = ADOS_social_sorted[whole2male_idx_ABD2]
t = ADOS_behav_sorted[whole2male_idx_ABD2]
u = age_sorted[whole2male_idx_ABD2]
v = iq_sorted[whole2male_idx_ABD2]

idx1 = ASD_DMN_pc_sub1_idx
idx2 = ASD_DMN_pc_sub2_idx
idx3 = ASD_DMN_pc_sub3_idx

In [ ]:
import statsmodels as sm
from statsmodels.stats.multitest import multipletests

ADOS_Total = []
ADOS_Comm = []
ADOS_Social = []
ADOS_Behav = []
age = []
iq = []

idx_list1 = [idx1, idx1, idx2] # idx3, idx3, idx1
idx_list2 = [idx2, idx3, idx3] # idx1, idx2, idx2

for i in range(3):
    a = idx_list1[i] # idx1 idx1 idx2
    b = idx_list2[i] # idx2 idx3 idx3

    [s_0,p_0] = sc.stats.ttest_ind(x[a][x[a]>=0], x[b][x[b]>=0], equal_var=False, axis=0)
    [s_1,p_1] = sc.stats.ttest_ind(y[a][y[a]>=0], y[b][y[b]>=0], equal_var=False, axis=0) 
    [s_2,p_2] = sc.stats.ttest_ind(z[a][z[a]>=0], z[b][z[b]>=0], equal_var=False, axis=0) 
    [s_3,p_3] = sc.stats.ttest_ind(t[a][t[a]>=0], t[b][t[b]>=0], equal_var=False, axis=0)
    [s_4,p_4] = sc.stats.ttest_ind(u[a][u[a]>=0], u[b][u[b]>=0], equal_var=False, axis=0)
    [s_5,p_5] = sc.stats.ttest_ind(v[a][v[a]>=0], v[b][v[b]>=0], equal_var=False, axis=0)

    p_0_fdr = sm.stats.multitest.multipletests(p_0,alpha=0.05,method='fdr_bh')
    p_1_fdr = sm.stats.multitest.multipletests(p_1,alpha=0.05,method='fdr_bh')
    p_2_fdr = sm.stats.multitest.multipletests(p_2,alpha=0.05,method='fdr_bh')
    p_3_fdr = sm.stats.multitest.multipletests(p_3,alpha=0.05,method='fdr_bh')
    p_4_fdr = sm.stats.multitest.multipletests(p_4,alpha=0.05,method='fdr_bh')
    p_5_fdr = sm.stats.multitest.multipletests(p_5,alpha=0.05,method='fdr_bh')

    ADOS_Total.append(p_0_fdr[1][0])
    ADOS_Comm.append(p_1_fdr[1][0])
    ADOS_Social.append(p_2_fdr[1][0])
    ADOS_Behav.append(p_3_fdr[1][0])
    age.append(p_4_fdr[1][0])
    iq.append(p_5_fdr[1][0])

result = {'ADOS_Total' : [np.round(i,4) for i in ADOS_Total], 
          'ADOS_Comm' : [np.round(i,4) for i in ADOS_Comm], 
          'ADOS_Social' : [np.round(i,4) for i in ADOS_Social],
          'ADOS_Behav' : [np.round(i,4) for i in ADOS_Behav],
          'Age' : [np.round(i,4) for i in age],
          'IQ' : [np.round(i,4) for i in iq]}
                   
[f_age,p_age] = sc.stats.f_oneway(np.array(u)[idx1], np.array(u)[idx2], np.array(u)[idx3])
[f_iq,p_iq] = sc.stats.f_oneway(np.array(v)[idx1][np.array(v)[idx1]>=0], np.array(v)[idx2], np.array(v)[idx3])
    
for key, val in result.items():
    print(f'{key}    :    {val}')    
    
print(f'Age F : {np.round(f_age,3)}, p : {np.round(p_age,3)}')
print(f'IQ F : {np.round(f_iq,3)}, p : {np.round(p_iq,3)}')

In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

FRF_subtype_label = np.zeros(len(ASD_label_idx))
FRF_subtype_label[:] = np.nan
FRF_subtype_label[idx1] =1 # 2
FRF_subtype_label[idx2] =2 # 3
FRF_subtype_label[idx3] =3 # 1

FRF_subtype_label

# y = ADOS_social_sorted # ADOS_Total_sorted ADOS_comm_sorted ADOS_social_sorted ADOS_behav_sorted

score_name = ['ADOS Total','ADOS Communication','ADOS Social','ADOS Behavior', 'Age', 'IQ']

for i, score in enumerate([ADOS_Total_sorted, ADOS_comm_sorted, ADOS_social_sorted, ADOS_behav_sorted, age_sorted, iq_sorted]):

    df = DataFrame([score[:len(ASD_label_idx)][score[:len(ASD_label_idx)]>=0], FRF_subtype_label[[score[:len(ASD_label_idx)]>=0]]]) 
    df = df.T
    df.columns = [score_name[i],'ASD Subtype']

    plt.figure(figsize = (7,7))
    sns.set(style = 'whitegrid', font_scale=2.5)
    sns.boxplot(x = 'ASD Subtype' , y = df.columns[0], data = df, palette = 'gist_yarg', width=0.5)
    sns.swarmplot(x = 'ASD Subtype' , y = df.columns[0], data = df, color = 'k', size = 4)
    plt.xticks([0,1,2],['ASD 1','ASD 2', 'ASD 3'], fontsize=40)
    plt.xlabel('Subtype', fontsize=40)
    plt.ylabel(score_name[i], fontsize=40)


In [ ]:
# ADOS Calibration score

demo_cali = pd.read_excel(join(path_data, 'abide_abideII_calibration.xlsx'), sheet_name='n=103', skiprows=0)

FileID_cali = demo_cali['ID']
ADOS_Total_cali = demo_cali['ADOS_T_CSS']

score = []
for i in sub_list:
    score.append(ADOS_Total_cali[np.where(FileID_cali == i)[0][0]])
    
score = np.array(score)

y = score[ASD_index]

df = DataFrame([y[:len(ASD_label_idx)][y[:len(ASD_label_idx)]>0], FRF_subtype_label[[y[:len(ASD_label_idx)]>0]]]) # cluster_labels[ADOS_Total_idx_sorted] SRS_exist_idx_sorted
df = df.T
df.columns = ['ADOS Total Calibration','ASD Subtype']

plt.figure(1,(7,7))
sns.set(style = 'whitegrid', font_scale=2.5)
sns.boxplot(x = 'ASD Subtype' , y = df.columns[0], data = df, palette = 'gist_yarg', width=0.5)
sns.swarmplot(x = 'ASD Subtype' , y = df.columns[0], data = df, color = 'k', size = 4)

In [412]:
# save ABD2 symptom severity score

ABD2_Score = DataFrame([ADOS_Total_sorted[:len(ASD_label_idx)], ADOS_comm_sorted[:len(ASD_label_idx)], ADOS_social_sorted[:len(ASD_label_idx)], ADOS_behav_sorted[:len(ASD_label_idx)], FRF_subtype_label[:len(ASD_label_idx)]]).T
ABD2_Score.columns = ['ADOS Total','ADOS Communication','ADOS Social','ADOS Behavior', 'ASD Subtype']
ABD2_Score
# ABD2_Score.to_excel(join(path_data,'ABD2_Score_male_revision.xlsx'), sheet_name = 'Sheet1', header = True)

# Compare symptom severity between discovery and replication

In [413]:
ABD1_Score = pd.read_excel(join(path_data, 'ABD1_Score_revision.xlsx'), sheet_name='Sheet1', skiprows=0)
ABD2_Score = pd.read_excel(join(path_data, 'ABD2_Score_male_revision.xlsx'), sheet_name='Sheet1', skiprows=0)

ABD1_Score['Dataset'] = 'Discovery'
ABD2_Score['Dataset'] = 'Replication'

In [ ]:
for score_name in ['ADOS Total','ADOS Communication','ADOS Social','ADOS Behavior', 'SRS']:
    
    if score_name != 'SRS':
        print(f'@@ {score_name} @@')

        df_ABD1 = DataFrame([ABD1_Score[score_name], ABD1_Score['ASD Subtype'], ABD1_Score['Dataset']]).T
        df_ABD2 = DataFrame([ABD2_Score[score_name], ABD2_Score['ASD Subtype'], ABD2_Score['Dataset']]).T

        df_ABD12 = pd.concat([df_ABD1, df_ABD2], axis=0)

        df_plot = df_ABD12[df_ABD12[score_name]>0]

        plt.figure(figsize=(10,7))
        sns.set(style = 'whitegrid', font_scale=1.5)
        g = sns.pointplot(x = 'ASD Subtype' , y = score_name, hue = 'Dataset', data = df_plot,  palette = "gray", plot_kws=dict(alpha=1))
        plt.xticks([0,1,2],['ASD 1','ASD 2', 'ASD 3'], fontsize=40)
        plt.xlabel('Subtype', fontsize = 40)
        plt.ylabel(score_name, fontsize = 40)
        plt.legend(fontsize=30)
        # plt.ylim([0,8])
        # plt.yticks([0,2,4,6])
        plt.setp(g.collections, alpha = 1)
        plt.setp(g.lines[5:8], alpha=.5)  
        # sns.swarmplot(x = 'ASD Subtype' , y = 'ADOS Communication', data = df_plot, color = '.25')

        is_ABD1 = df_plot['Dataset'] == 'Discovery'
        is_ABD2 = df_plot['Dataset'] == 'Replication'
        is_sub1 = df_plot['ASD Subtype'] == 1
        is_sub2 = df_plot['ASD Subtype'] == 2
        is_sub3 = df_plot['ASD Subtype'] == 3

        chi_ob=[]
        chi_expect=[]

        for num in [is_sub1, is_sub2, is_sub3]:
            print('p value : ',np.round(sc.stats.ttest_ind(df_plot[num & is_ABD1][score_name],df_plot[num & is_ABD2][score_name])[1], 4))
            print('ABIDE 1 score : ',np.round(df_plot[num & is_ABD1][score_name].mean(),2),' +- ', np.round(df_plot[num & is_ABD1][score_name].std(),2))
            print('ABIDE 2 score : ',np.round(df_plot[num & is_ABD2][score_name].mean(),2),' +- ', np.round(df_plot[num & is_ABD2][score_name].std(),2))
            print(' ')
            chi_expect.append(df_plot[num & is_ABD1][score_name].mean())
            chi_ob.append(df_plot[num & is_ABD2][score_name].mean())

        # Chi square test between ABD1 and ABD2
        print(np.round(chi_ob,2))
        print(np.round(chi_expect,2))
        print('')
        print('Chi Square')
        print('static : ', np.round(sc.stats.chisquare(chi_ob, f_exp=chi_expect, axis=None, ddof = 0).statistic,3))
        print('p value : ',np.round(sc.stats.chisquare(chi_ob, f_exp=chi_expect, axis=None, ddof = 0).pvalue,3))
        print(' ')
        
    else:
        print(f'@@ {score_name} @@')

        df_ABD1 = DataFrame([ABD1_Score[score_name], ABD1_Score['ASD Subtype'], ABD1_Score['Dataset']]).T
        df_plot = df_ABD1[df_ABD1[score_name]>0]
        
        is_ABD1 = df_plot['Dataset'] == 'Discovery'
        is_sub1 = df_plot['ASD Subtype'] == 1
        is_sub2 = df_plot['ASD Subtype'] == 2
        is_sub3 = df_plot['ASD Subtype'] == 3
        
        for num in [is_sub1, is_sub2, is_sub3]:
            print('ABIDE 1 score : ',np.round(df_plot[num & is_ABD1][score_name].mean(),2),' +- ', np.round(df_plot[num & is_ABD1][score_name].std(),2))

# Statistical analysis

In [496]:
#ttest
def subtype_ranksum(a,b):
    import statsmodels as sm
    from statsmodels.stats.multitest import multipletests
    
    input_a = a # [X, Y, Z, X, X, Y, W]
    input_b = b # [S, S, S, Y, Z, Z, R]

    s_1 = []
    p_1 = []

    for col in range(input_a.shape[1]):
        [s,p] = sc.stats.ranksums(input_a[:,col],input_b[:,col])
        s_1.append(s)
        p_1.append(p)
    
#     [s_1,p_1] = sc.stats.ttest_ind(input_a, input_b, axis=0, equal_var=False)
    
    s_1 = np.array(s_1)
    p_1 = np.array(p_1)

    # Calculate Effect size r
    effect_r = s_1/np.sqrt(len(input_a)+len(input_b))

    p_1_fdr_05 = sm.stats.multitest.multipletests(p_1,alpha=0.2,method='fdr_bh')
    p_1_fdr_01 = sm.stats.multitest.multipletests(p_1,alpha=0.01,method='fdr_bh')
    p_1_fdr_001 = sm.stats.multitest.multipletests(p_1,alpha=0.001,method='fdr_bh')

    print('FDR uncorrected : ', np.where(p_1 <0.05)[0], np.where(p_1 <0.05)[0].shape )
    print('')
    print(np.where(p_1_fdr_05[0]==True),'\n', np.where(p_1_fdr_05[0]==True)[0].shape)
    print('FDR 0.05 : ', p_1_fdr_05[1][np.where(p_1_fdr_05[0]==True)[0]])
    print('')
#     print(np.where(p_1_fdr_01[0]==True),'\n', np.where(p_1_fdr_01[0]==True)[0].shape)
#     print('FDR 0.01 : ', p_1_fdr_01[1][np.where(p_1_fdr_01[0]==True)[0]])
#     print('')
#     print(np.where(p_1_fdr_001[0]==True),'\n', np.where(p_1_fdr_001[0]==True)[0].shape)
#     print('FDR 0.001 : ', p_1_fdr_001[1][np.where(p_1_fdr_001[0]==True)[0]])

    sign_idx = np.where(p_1_fdr_05[0]==True)[0]
    sign_idx_uncorr = np.where(p_1 <0.05)[0]
    
    ttest_tval = np.zeros(360)
    input_idx = sign_idx 
    ttest_tval[input_idx] = effect_r[input_idx] 
       
    return ttest_tval, sign_idx_uncorr

# Prepare visualization

def ROI_visualization(input_stat,stats = 'Ttest'):

    import vtk
    from vtk import vtkPolyDataNormals
    from brainspace.mesh.mesh_io import read_surface
    from brainspace.mesh.mesh_operations import combine_surfaces
    from brainspace.utils.parcellation import reduce_by_labels
    from brainspace.vtk_interface import wrap_vtk, serial_connect

    template_path = join(atlas_path,"MMP")
    template_L = "L.very_inflated_MSMAll.10k_fs_LR.surf.gii" # S900.L.midthickness_MSMAll.10k_fs_LR.surf.gii # L.very_inflated_MSMAll.10k_fs_LR.surf.gii
    template_R = "R.very_inflated_MSMAll.10k_fs_LR.surf.gii" # S900.R.midthickness_MSMAll.10k_fs_LR.surf.gii # R.very_inflated_MSMAll.10k_fs_LR.surf.gii

    surfs = [None] * 2
    
    surfs[0] = read_surface(join(template_path,template_L)) # Z:/hschoi/backup/hschoi/template/MMP/S900.L.midthickness_MSMAll.10k_fs_LR.surf.gii
    nf = wrap_vtk(vtkPolyDataNormals, splitting=False, featureAngle=0.1)
    surf_lh = serial_connect(surfs[0], nf)

    surfs[1] = read_surface(join(template_path,template_R)) # Z:/hschoi/backup/hschoi/template/MMP/S900.R.midthickness_MSMAll.10k_fs_LR.surf.gii
    nf = wrap_vtk(vtkPolyDataNormals, splitting=False, featureAngle=0.1)
    surf_rh = serial_connect(surfs[1], nf)

    # Visualization

    from brainspace.datasets import load_group_fc, load_parcellation, load_conte69
    from brainspace.gradient import GradientMaps
    from brainspace.plotting import plot_hemispheres
    from brainspace.utils.parcellation import map_to_labels

    atlas = np.load(join(template_path,"MMP.10k_fs_LR.npy"))
    labeling = atlas 
    conn_matrix = input_stat 
    mask = labeling != 0
    grad = map_to_labels(conn_matrix, labeling, mask=mask, fill=np.nan) # fill = np.nan fill = 0

    if stats == 'ANOVA':
        plot_hemispheres(surf_lh, surf_rh, array_name=grad, size=(1300, 300),
                         color_bar=True, cmap='Reds_r', zoom=1.25, nan_color=(0,0,0,1) ,color_range = (0,0.5)   ) 
                                                                                                                       
    elif stats == 'Ttest':
        plot_hemispheres(surf_lh, surf_rh, array_name=grad, size=(1300, 300),
                         color_bar=True, cmap='seismic', zoom=1.25, nan_color=(0,0,0,1) ,color_range = (-1,1) , view = 'dorsal'  )  # ventral dorsal
                                                                                                                

In [ ]:
# ttest
X = ASD_DMN_pc_sub1
Y = ASD_DMN_pc_sub2 
Z = ASD_DMN_pc_sub3
W = TD_DMN_pc_sub1
R = TD_DMN_pc_sub2 
V = TD_DMN_pc_sub3

S = np.concatenate((W,R))
U = np.concatenate((X,Y,Z))

print('T-test')
print('ASD sub1')
effect_r_ASD1_TD, ASD1_TD_idx = subtype_ranksum(X,S)
print('ASD sub2')
effect_r_ASD2_TD, ASD2_TD_idx = subtype_ranksum(Y,S)
print('ASD sub3')
effect_r_ASD3_TD, ASD3_TD_idx = subtype_ranksum(Z,S)
print('ASD sub12')
effect_r_ASD12, ASD12_idx = subtype_ranksum(X,Y)
print('ASD sub13')
effect_r_ASD13, ASD13_idx = subtype_ranksum(X,Z)
print('ASD sub23')
effect_r_ASD23, ASD23_idx = subtype_ranksum(Y,Z)
print('TD sub12')
effect_r_TD12, TD12_idx = subtype_ranksum(W,R)

# print('')
# print('ANOVA')
# ANOVA_eta = subtype_kruskal(X,Y,Z)

# ROI Visulaization

In [498]:
# ROI_visualization(ANOVA_eta, stats = 'ANOVA')
ROI_visualization(effect_r_ASD1_TD, stats = 'Ttest')

# Make file for Neurosynth

In [95]:
def file_for_Neurosynth(input_stats, subtype_name = 'ASD1'):

    atlas = np.load(join(template_path,"MMP.10k_fs_LR.npy"))

    atlas_L = atlas[:10242]
    atlas_R = atlas[10242:]

    # atlas.max()

    atlas_sig=np.zeros(len(atlas))
    for i in input_stats:                       # ASD12_idx ASD13_idx ASD23_idx ASD1_TD12_idx ASD2_TD12_idx ASD3_TD12_idx
        print(i+1, ' ', end='', flush=True)

        atlas_sig[np.where(atlas==i+1)[0]] = cohens_d[i]

    atlas_sig_L = atlas_sig[:10242]
    atlas_sig_R = atlas_sig[10242:]

    np.save(join(path_gii_data,f'L.MMP.10k.ef_r_{subtype_name}_TD_ABD2.npy'),atlas_sig_L) # atlas
    np.save(join(path_gii_data,f'R.MMP.10k.ef_r_{subtype_name}_TD_ABD2.npy'),atlas_sig_R)

In [ ]:
file_for_Neurosynth(ASD1_ranksum_idx, subtype_name = 'ASD1')
file_for_Neurosynth(ASD2_ranksum_idx, subtype_name = 'ASD2')
file_for_Neurosynth(ASD3_ranksum_idx, subtype_name = 'ASD3')